In [150]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle
import mariadb
import uuid
import json
import random

In [151]:
data = pd.read_excel('RESOURCES.xlsx')
data2 = pd.read_excel('SITES.xlsx')
dataMerged = merged_df = pd.merge(data, data2, on="Site Name", how="left", suffixes=("_RESOURCES", "_SITES"))
dataMerged["email"] = dataMerged["Site Name"].str.replace(" ", "").str.lower() + "@voorbeeld.nl"
dataMerged["Unit of Measure"] = dataMerged["Unit of Measure"].str.replace(" ", "")
dataMerged["Unit of Measure"] = dataMerged["Unit of Measure"].str.replace("Money(€)", "MoneyEUR")

dataMerged["Unit of Measure"].unique()

array(['Number', 'Hectares', 'Hours', 'Tonnes', 'Kilograms', 'Litres',
       'CubicMetres', 'SquareMetres', 'MW', 'KWh', 'MoneyEUR'],
      dtype=object)

## INSERT LOCATIONS

In [152]:
# Filtering out unneeded columns
# location = dataMerged[["Street and Number_SITES", "Postal Code_SITES", "Latitude", "Longitude", "Site Name", "City_SITES"]]
dataMerged['Location_Id'] = [str(uuid.uuid4()) for _ in range(len(dataMerged))]
dataMerged = dataMerged.rename(columns={"Street and Number_SITES": "Address"})

# Display the filtered DataFrame
print(dataMerged)

# Database connection details
db_config = {
    "user": "root",
    "password": "None58-DB",
    "host": "localhost",
    "port": 3306,
    "database": "SymbioDb"
}

# Connect to MariaDB
try:
    conn = mariadb.connect(**db_config)
    cursor = conn.cursor()

    # SQL query for inserting data
    insert_query = """
    INSERT INTO locations (Id, Address, PostalCode, Latitude, Longitude, City, Country)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """

    # Iterate through each row in the DataFrame
    for _, row in dataMerged.iterrows():

        # Execute the query with values
        cursor.execute(insert_query, (
            row["Location_Id"],
            row["Address"],
            row["Postal Code_SITES"],
            row["Latitude"],
            row["Longitude"],
            row["City_SITES"],
            'Netherlands'
        ))

    # Commit the transaction
    conn.commit()
    print(f"{cursor.rowcount} rows inserted successfully.")

except mariadb.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        cursor.close()
        conn.close()

                           Resource Name  \
0     1x vrije vrachtwagen met chauffeur   
1     1x vrije vrachtwagen met chauffeur   
2                         3 ha zandgrond   
3               3D Metaalprintcapaciteit   
4               3D Metaalprintcapaciteit   
...                                  ...   
1555                          zoet water   
1556               zonne- en windenergie   
1557                       Zonne-energie   
1558                       Zonne-energie   
1559               zout, natriumchloride   

                                  Description_RESOURCES  \
0     Een vrije vrachtwagen, incl. chauffeur, voor t...   
1     Een vrije vrachtwagen, incl. chauffeur, voor t...   
2        3 ha zandgrond landbouw kwaliteit in Strijbeek   
3     3D Metaalprintcapaciteit, volume 250x250X300mm...   
4     3D Metaalprintcapaciteit, volume 250x250X300mm...   
...                                                 ...   
1555                            zoet water voor koeling   

## INSERT COMPANIES

In [153]:
dataMerged['Company_Id'] = [str(uuid.uuid4()) for _ in range(len(dataMerged))]
dataMerged['Description_SITES'] = dataMerged['Description_SITES'].fillna("No Description")
# companies = dataMerged[['Company_Id', 'Site Name', 'Description_SITES', 'NACE Code', 'email', 'Location_Id']]


# nan_locations = companies.isna()

# # Display the DataFrame showing True where values are NaN
# print("Locations of NaN values:")
# print(nan_locations)

# # To get a summary of columns with NaN values
# nan_summary = companies.isna().sum()
# print("\nSummary of NaN values per column:")
# print(nan_summary)

# Connect to MariaDB
try:
    conn = mariadb.connect(**db_config)
    cursor = conn.cursor()

    # SQL query for inserting data
    insert_query = """
    INSERT INTO companies (Id, Name, Description, NaceCode, Email, Phonenumber, LocationId, BundleId)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """

    # Iterate through each row in the DataFrame
    for _, row in dataMerged.iterrows():

        # Execute the query with values
        cursor.execute(insert_query, (
            row["Company_Id"],
            row["Site Name"],
            row["Description_SITES"],
            row["NACE Code"],
            row["email"],
            "0612345678",
            row["Location_Id"],
            None
        ))

    # Commit the transaction
    conn.commit()
    print(f"{cursor.rowcount} rows inserted successfully.")

except mariadb.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        cursor.close()
        conn.close()


0 rows inserted successfully.


## INSERT Products

In [154]:
dataMerged[["Category 1", "Category 2", "Category 3"]] = dataMerged[["Category 1", "Category 2", "Category 3"]].replace({"": "Unkwnown", None: "Unkwnown", " ": "Unkwnown"})
dataMerged["Category 1"] = dataMerged["Category 1"].str.replace(r"^\d+\s+", "", regex=True)
dataMerged["Category 2"] = dataMerged["Category 2"].str.replace(r"^\d+\s+\d+\s+", "", regex=True)
dataMerged["Category 3"] = dataMerged["Category 3"].str.replace(r"^\d+\s+\d+\s+\d+\s+", "", regex=True)

dataMerged['Product_Id'] = [str(uuid.uuid4()) for _ in range(len(dataMerged))]
dataMerged["Valid From"] = pd.to_datetime(dataMerged["Valid From"]).dt.strftime("%Y-%m-%d")
dataMerged["Valid To"] = pd.to_datetime(dataMerged["Valid To"]).dt.strftime("%Y-%m-%d")
dataMerged["Create Date"] = pd.to_datetime(dataMerged["Create Date"]).dt.strftime("%Y-%m-%d")
dataMerged['Description_RESOURCES'] = dataMerged['Description_RESOURCES'].fillna("No Description")
dataMerged["CategoryList"] = dataMerged[["Category 1", "Category 2", "Category 3"]].apply(lambda row: list(row), axis=1)
dataMerged["CategoryList"] = dataMerged["CategoryList"].apply(json.dumps)
dataMergedHave = dataMerged[dataMerged["Resource Type"] == "Have"]
# Connect to MariaDB
try:
    conn = mariadb.connect(**db_config)
    cursor = conn.cursor()

    # SQL query for inserting data
    insert_query = """
    INSERT INTO products (Id, Name, Description, SupplyType, Categories, ValidFrom, ValidTo, CreatedOn, CompanyId, BundleId, IsBundled, BundlingAllowed)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    # Iterate through each row in the DataFrame
    for _, row in dataMergedHave.iterrows():

        # Execute the query with values
        cursor.execute(insert_query, (
            row["Product_Id"],
            row["Resource Name"],
            row["Description_RESOURCES"],
            row["Supply Type"],
            row["CategoryList"],
            row["Valid From"],
            row["Valid To"],
            row["Create Date"],
            row["Company_Id"],
            None,
            False,
            True



        ))

    # Commit the transaction
    conn.commit()
    print(f"{cursor.rowcount} rows inserted successfully.")

except mariadb.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        cursor.close()
        conn.close()


0 rows inserted successfully.


## INSERT MATERIALS

In [155]:
dataMergedHave['Material_Id'] = [str(uuid.uuid4()) for _ in range(len(dataMergedHave))]

# Connect to MariaDB
try:
    conn = mariadb.connect(**db_config)
    cursor = conn.cursor()

    # SQL query for inserting data
    insert_query = """
    INSERT INTO materials (Id, Name, Description, AvailableQuantity, UnitOfMeasure, ProductId)
    VALUES (?, ?, ?, ?, ?, ?)
    """

    # Iterate through each row in the DataFrame
    for _, row in dataMergedHave.iterrows():

        # Execute the query with values
        cursor.execute(insert_query, (
            row["Material_Id"],
            row["Resource Name"],
            row["Description_RESOURCES"],
            row["Available Quantity"],
            row["Unit of Measure"],
            row["Product_Id"],
        ))

    # Commit the transaction
    conn.commit()
    print(f"{cursor.rowcount} rows inserted successfully.")

except mariadb.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        cursor.close()
        conn.close()



0 rows inserted successfully.


C:\Users\bende\AppData\Local\Temp\ipykernel_63880\3161010882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataMergedHave['Material_Id'] = [str(uuid.uuid4()) for _ in range(len(dataMergedHave))]


## Insert Preferences

In [156]:

# Check if 'user_Id' column exists in the DataFrame
if "user_Id" not in dataMerged.columns:
    try:
        # Connect to MariaDB
        conn = mariadb.connect(**db_config)
        cursor = conn.cursor()

        # Query to retrieve Id and email from aspnetusers
        query = "SELECT Id, email FROM aspnetusers"
        cursor.execute(query)

        # Fetch results and create a DataFrame
        aspnetusers_data = cursor.fetchall()
        aspnetusers_df = pd.DataFrame(aspnetusers_data, columns=["user_Id", "email"])


        # Merge aspnetusers DataFrame with dataMerged on the email column
        dataMerged = pd.merge(aspnetusers_df, dataMerged, on="email", how="left")

        print("user_Id column added successfully.")
    except mariadb.Error as e:
        print(f"Error: {e}")

    finally:
        if conn:
            cursor.close()
            conn.close()
else:
    print("The 'user_Id' column already exists in the DataFrame.")

    
dataMerged["Keywords"] = dataMerged["Keywords"].fillna("")
dataMergedWant = dataMerged[dataMerged["Resource Type"] == "Want"]


random.seed(42)

dataMergedWant["CategoryList2"] = dataMerged[["Category 1", "Category 2", "Category 3"]].apply(lambda row: list(row), axis=1)
# dataMerged["CategoryList"] = dataMerged["CategoryList"].apply(json.dumps)


# Group by 'user_Id' and apply aggregation
grouped = dataMergedWant.groupby("user_Id").agg({
    "CategoryList2": lambda x: ", ".join(set(item for sublist in x for item in sublist)),  # Flatten and deduplicate, then CSV
    "Keywords": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Unit of Measure": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Supply Type": lambda x: ", ".join(set(x)),  # Deduplicate and convert to CSV
    "Available Quantity": ["min", "max"],  # Minimum and maximum quantity
    "Valid From": "min",  # Smallest date value
    "Valid To": "max",  # Largest date value
    "Site Name": "first",  # Use the first occurrence of the Site Name
}).reset_index()

# Flatten column names for "Available Quantity"
grouped.columns = ["_".join(col).strip("_") if isinstance(col, tuple) else col for col in grouped.columns]

# Rename columns for clarity
grouped.rename(columns={
    "Available Quantity_min": "Minimum Quantity",
    "Available Quantity_max": "Maximum Quantity",
    "Supply Type_<lambda>": "Supply Type",
    "Keywords_<lambda>": "Keywords",
    "CategoryList2_<lambda>": "CategoryList",
    "Valid From_min": "Valid From",
    "Valid To_max": "Valid To",
    "Site Name_first" : "Site Name",
    "Unit of Measure_<lambda>" : "Unit of Measure"

}, inplace=True)

grouped['Preference_Id'] = [str(uuid.uuid4()) for _ in range(len(grouped))]



# Sort by site name to ensure same value for max distance is given 
random.seed(42)
grouped = grouped.sort_values(by="Site Name")

grouped.head()

# Connect to MariaDB
try:
    conn = mariadb.connect(**db_config)
    cursor = conn.cursor()

    # SQL query for inserting data
    insert_query = """
    INSERT INTO userpreferences (Id, PreferredCategories, PreferredKeywords, MaxSearchRadiusKM, MinimumAvailableQuantity, MaximumAvailableQuantity, PreferredSupplyType, PreferredUnitOfMeasures, PreferredValidFrom, PreferredValidTo, UserId)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    # Iterate through each row in the DataFrame
    for _, row in grouped.iterrows():

        # Execute the query with values
        cursor.execute(insert_query, (
            row["Preference_Id"],
            row["CategoryList"],
            row["Keywords"],
            str(random.randint(10, 100)),
            row["Minimum Quantity"],
            row["Maximum Quantity"],
            row["Supply Type"],
            row["Unit of Measure"],
            row["Valid From"],
            row["Valid To"],
            row["user_Id"],




        ))

    # Commit the transaction
    conn.commit()
    print(f"{cursor.rowcount} rows inserted successfully.")

except mariadb.Error as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        cursor.close()
        conn.close()

user_Id column added successfully.
0 rows inserted successfully.


C:\Users\bende\AppData\Local\Temp\ipykernel_63880\2776275614.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataMergedWant["CategoryList2"] = dataMerged[["Category 1", "Category 2", "Category 3"]].apply(lambda row: list(row), axis=1)


In [4]:
# Imports 
import pandas as pd
import mariadb
import uuid
import json
import random
dataMerged = pd.read_excel('C:\dev\Jaar4\Periode2\CapstoneProject\AI\DATA_MERGED.xlsx')

db_config = {
    "user": "root",
    "password": "None58-DB",
    "host": "localhost",
    "port": 3306,
    "database": "SymbioDb"
}

# Check if 'user_Id' column exists in the DataFrame
if "user_Id" not in dataMerged.columns:
    try:
        # Connect to MariaDB
        conn = mariadb.connect(**db_config)
        cursor = conn.cursor()

        # Query to retrieve Id and email from aspnetusers
        query = "SELECT Id, email FROM aspnetusers"
        cursor.execute(query)

        # Fetch results and create a DataFrame
        aspnetusers_data = cursor.fetchall()
        aspnetusers_df = pd.DataFrame(aspnetusers_data, columns=["user_Id", "email"])


        # Merge aspnetusers DataFrame with dataMerged on the email column
        dataMerged = pd.merge(aspnetusers_df, dataMerged, on="email", how="left")

        print("user_Id column added successfully.")
    except mariadb.Error as e:
        print(f"Error: {e}")

    finally:
        if conn:
            cursor.close()
            conn.close()
else:
    print("The 'user_Id' column already exists in the DataFrame.")

    
dataMerged["Keywords"] = dataMerged["Keywords"].fillna("")
dataMergedWant = dataMerged[dataMerged["Resource Type"] == "Want"]


random.seed(42)

# dataMergedWant["CategoryList2"] = dataMerged[["Category 1", "Category 2", "Category 3"]].apply(lambda row: list(row), axis=1)
# dataMerged["CategoryList"] = dataMerged["CategoryList"].apply(json.dumps)


# Group by 'user_Id' and apply aggregation
grouped = dataMergedWant.groupby("user_Id").agg({
    "Category 1": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Category 2": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Category 3": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Keywords": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Unit of Measure": lambda x: ", ".join(set(x)),  # Flatten and deduplicate, then CSV
    "Supply Type": lambda x: ", ".join(set(x)),  # Deduplicate and convert to CSV
    "Available Quantity": ["min", "max"],  # Minimum and maximum quantity
    "Valid From": "min",  # Smallest date value
    "Valid To": "max",  # Largest date value
    "Site Name": "first",  # Use the first occurrence of the Site Name
}).reset_index()

# Flatten column names for "Available Quantity"
grouped.columns = ["_".join(col).strip("_") if isinstance(col, tuple) else col for col in grouped.columns]

# Rename columns for clarity
grouped.rename(columns={
    "Available Quantity_min": "Minimum Quantity",
    "Available Quantity_max": "Maximum Quantity",
    "Supply Type_<lambda>": "Supply Type",
    "Keywords_<lambda>": "Keywords",
    "CategoryList2_<lambda>": "CategoryList",
    "Valid From_min": "Valid From",
    "Valid To_max": "Valid To",
    "Site Name_first" : "Site Name",
    "Unit of Measure_<lambda>" : "Unit of Measure",
    "Category 1_<lambda>": "Category 1",
    "Category 2_<lambda>": "Category 2",
    "Category 3_<lambda>": "Category 3",

}, inplace=True)

grouped['Preference_Id'] = [str(uuid.uuid4()) for _ in range(len(grouped))]



# Sort by site name to ensure same value for max distance is given 
random.seed(42)
grouped = grouped.sort_values(by="Site Name")

grouped.head()


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\bende\AppData\Local\Temp\ipykernel_117428\4152989216.py:7: SyntaxWarning: invalid escape sequence '\d'
  dataMerged = pd.read_excel('C:\dev\Jaar4\Periode2\CapstoneProject\AI\DATA_MERGED.xlsx')


user_Id column added successfully.


,user_Id,Category 1,Category 2,Category 3,Keywords,Unit of Measure,Supply Type,Minimum Quantity,Maximum Quantity,Valid From,Valid To,Site Name,Preference_Id
48,4f627d9c-2148-444c-9fed-08be27723013,"Capacity, Energy, Inorganics, Wood, Food & Agr...","Green Waste, Laboratories, Gases, Processed Wo...","Other Food Sludges & Wastes, Unspecified, Carb...",", Lelystad, MDF, diverse fracties, snijafval, ...","Kilograms, Hours, Litres, CubicMetres, MW, Tonnes","Batch, Continuous",5.0,600000.0,2019-06-14,2024-06-17,4evergreen - Steenbergen - Head office,00e8a67b-1ac8-4472-b326-316f736288ac
106,c4bef250-ec27-4ccd-9873-02afa91cc500,"Expertise, Capacity, Energy, Water, Logistics,...","Prepared Food Waste, Commercial, Green Waste, ...","Other Food Sludges & Wastes, Unspecified, Biog...",", laagwaardige restwarmte","Number, CubicMetres, MW, Tonnes","Batch, Continuous",15.0,1000000.0,2018-11-26,2023-11-28,Aben Green Energy - Westdorpe - Head office,875ca353-189c-423a-a549-128aa82a95d0
116,d6b64e10-1eb0-4a30-a4a3-37265915e6d8,"Expertise, Capacity, Energy, Water, Wood","Product, Storage, Logistics, Drainage Water, M...","Steam, Other Logistics Management, Industrial ...",", onbekende hoeveelheid continu 24/7, Proceswa...","Number, CubicMetres, MW",Continuous,15.0,99.0,2019-11-08,2024-11-15,Agristo - Tilburg - HO,b690ba98-5b58-4e78-b332-830675329b85
70,6bc2d35e-cba2-4c34-8441-f589797b5f5f,"Inorganics, Food & Agriculture, Water","Compost, Peat Etc, Unspecified, Nitrogen Compo...","Other Nitrogen Compounds, Compost, Unspecified",,"Kilograms, CubicMetres, Tonnes","Continuous, Batch",99.0,50000.0,2021-04-16,2026-04-16,Arie Bouman Tuinplanten,f491288e-fc0a-4702-aff8-a56cce2fb0d1
55,596ebd92-2ce3-490c-98d4-195d482267c3,Energy,Heat,Unspecified,,MW,Batch,99.0,99.0,2023-07-20,2028-07-20,Atlas Power and Gas,6a221020-c3b3-41d8-a1f4-f03a27e68384
